
# Intelligent Investment Decisions with Databricks

## AI is transforming portfolio management

In the competitive world of global finance, portfolio managers face immense pressure to make data-driven decisions in real time. But traditional methods of stock analysis are slow, manual, and prone to missed opportunities.

Modern financial institutions are turning to AI to revolutionize investment strategies, combining speed, accuracy, and personalization.

- But—manual processes and lack of real-time insights slow down decision-making!
- Portfolio managers need tools that combine speed and intelligence.


<br style="clear: both">

## Investment decisions are data + analysis.

Investment management is complex because delivering value isn't just about intuition or expertise.  
The process involves multiple steps—data collection, stock analysis, trend identification, risk assessment, and actionable recommendations.  
Stepping back, all these steps are powered by **data + intelligent tools**.

<img style="float:right;margin-top:10px" width="500px" src="/Workspace/Users/sarbani.maiti@databricks.com/BLOGATHON_Investment_Assistant/agent_app_img" />

<img src="/Workspace/Users/sarbani.maiti@databricks.com/BLOGATHON_Investment_Assistant/woman_image" style="float:left;" width="80px"> 
<h4 style="padding:10px 0px 0px 5px">Sophia, as a Portfolio Manager at CapitalCatalyst, needs an AI assistant to streamline her Investment analysis workflow:</h3>


- Identify customer preferences & company names
- Retrieve real-time stock data & trends
- Analyze stocks using historical & market data
- Provide actionable "Buy," "Sell," or "Hold" recommendations
- Justify recommendations with clear insights
- Highlight missing data for transparency**


**Sophia needs a cutting-edge AI Assistant which empowers portfolio managers to make smarter investment decisions. 

Let's see how can we build a AI driven Investment Assistant with Databricks Agent Framework.


# Building a domain specific AI Agent using Databricks Agent Framework

##  Investment Assistant Features

### Step 1: Identifying Company Names and Customer IDs
- The Agent can identify company names and customer IDs when explicitly provided by the user.
- This helps in tailoring investment recommendations to specific customer profiles and companies.

### Step 2: Retrieving Stock Ticker Symbols
- The Agent can retrieve stock ticker symbols from the internet if they are not provided by the user.
-  This ensures accurate identification of the stock in question for analysis.

### Step 3: Analyzing Stocks for Investment Recommendations
- The Agent uses various tools to analyze stocks and provide investment recommendations.
- This involves a detailed analysis using customer preferences and stock data.

###  Stock Analysis:
3.1. **Customer Investment Preferences**:
   - **Tool**: customer_investment_preferences
   - **Usage**: Retrieve customer risk appetite and other preferences for personalized recommendations.

3.2. **Current Stock Market Price**:
   - **Tool**: finance_stock_quote
   - **Usage**: Obtain the current market price and other relevant financial data for the stock.

3.3. **Stock Summary/Insights**:
   - **Tool**: finance_insight_api
   - **Usage**: Gather insights and summaries about the stocks performance and market position.

3.4. **Historical Stock Volume and Closing Price**:
   - **Tool**: get_historical_closing_price`
   - **Usage**: Extract historical data to understand stock trends and volatility.

### Step 4: Generating Final Recommendations
- The Agent generates recommendations based on the analysis, using specific numbers and insights.
- Recommendations are categorized as **Buy**, **Hold**, or **Sell**, with detailed justification.

### Step 5: Handling Missing Data
- If any required data is unavailable, the Agent explicitly states which data is missing.
- This ensures transparency and allows for recommendations based on available information.

### Step 6: Output Format
- The Agent follows a structured output format for recommendations.
- This includes bolded recommendations followed by detailed justification, ensuring clarity and comprehensiveness.

By following these steps, the Agent provides a comprehensive and personalized investment analysis, leveraging available tools and data to assist users in making informed investment decisions.

## This notebook will 

- Create the required Agent tools - functions
- Create the sample data & upload in Delta Tables


%md
## Create UC-Functions to access the yahoo finance API

Get access to :
https://www.financeapi.net/

Create your own API-KEY ( free key will have limitations, please check the dashboard, limk below)

Access to Yahoofinace dashboard : 
 https://www.financeapi.net/dashboard

 

Yahoofinance alternative 

https://site.financialmodelingprep.com/developer/docs#balance-sheet-statements-financial-statements


I have used one specfic researcher "Argus Research" to limit the number of tokens

In [0]:
# Drop the existing function if it exists to avoid conflicts
spark.sql('DROP FUNCTION IF EXISTS sarbanimaiti_catalog.agent_demo.finance_insight_api;')

# Create or replace the Unity Catalog function for financial insights
spark.sql('''
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.finance_insight_api (query STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Returns financial insights for a given ticker symbol.'
AS
$$
try:
    # Import requests module to make HTTP requests
    import requests
    # Define the API endpoint for financial insights
    url = "https://yfapi.net/ws/insights/v1/finance/insights"

    # Set the query parameter with the symbol provided to the function
    params = {"symbol": query}

    # Include your API key in the request headers for authentication
    headers = {
        'x-api-key': "Enter_your_key_here"
    }

    # Make a GET request to the API
    response = requests.request("GET", url, headers=headers, params=params)

    # Initialize an empty list to hold summaries
    summaries = []
    # Parse the JSON response
    data = response.json()
    # Control variable to ensure only one summary is added
    cntrol = 1

    # Iterate through the reports in the response
    for report in data['finance']['result']['reports']:
        # Check if the report is from Argus Research and control is 1
        if report['provider'] == "Argus Research" and cntrol ==1:
            # Append the summary to the summaries list
            summaries.append(report['summary'])
            # Update control to prevent adding more summaries
            cntrol = cntrol + 1

    # Return the list of summaries
    return summaries
except Exception as e:
    # Return an error message if an exception occurs
    return "Error calling YouTube Search API: {{e}}".format(e=str(e))
$$;
''')

DataFrame[]

In [0]:
# Test the finance_insight_api function with 'AAPL' as the parameter and collect the result
result = spark.sql("SELECT sarbanimaiti_catalog.agent_demo.finance_insight_api('AAPL')").collect()

In [0]:
result

[Row(sarbanimaiti_catalog.agent_demo.finance_insight_api(AAPL)='[\'Which of the market-dominant and so-called "Magnificent 7" stocks might be the best investments? During the first phase of unbridled AI enthusiasm, all of the Mag 7 were loved as their opportunity appeared to be almost unlimited. The market is now more skeptical toward perceived AI winners, which has led to a natural tendency to rank the group on their go-forward prospects. We see Amazon moving up in the ranks, while Nvidia continues to command the top spot.  All the Mag 7 names have something in common. They are all dominant in at least one core competency; and they have leveraged the immense cash flows thrown off by their core competencies to invest in and develop the generative AI opportunity. "Ranking" the Mag 7 on their intermediate-term prospects, therefore, involves assessing each company\\\'s ability to defend and nurture its core business while using that strength to gain share within the AI ecosystem. For this

In [0]:
# # Drop the existing function to avoid conflicts
# spark.sql('DROP FUNCTION IF EXISTS sarbanimaiti_catalog.finance_stock_quote.finance_api;')

# Define a new function in the Unity Catalog
spark.sql('''
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.finance_stock_quote (query STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Returns financial summary of ticker.'
AS
$$
try:
    # Import requests to enable HTTP calls
    import requests
    # API endpoint for fetching finance quotes
    url = "https://yfapi.net/v6/finance/quote"
    # Parameters for the API call, specifying the stock symbols
    params = {"symbols": query}
    # Headers for the API call, including the API key for authentication
    headers = {
        'x-api-key': "Enter_your_key_here"
    }
    # Execute the GET request with specified URL, headers, and parameters
    response = requests.request("GET", url, headers=headers, params=params)
    # Return the JSON response from the API
    return response.json()
except Exception as e:
    # Return an error message if an exception occurs
    return "Error calling YouTube Search API: {{e}}".format(e=str(e))
$$;
''')

DataFrame[]

In [0]:
# Test the SQL function to get the real time price , financial summary for the ticker 'AAPL' and collect the result
result = spark.sql("SELECT sarbanimaiti_catalog.agent_demo.finance_stock_quote('AAPL')").collect()

In [0]:
result

[Row(sarbanimaiti_catalog.agent_demo.finance_stock_quote(AAPL)="{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Nasdaq Real Time Price', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'USD', 'marketState': 'PRE', 'hasPrePostMarketData': True, 'firstTradeDateMilliseconds': 345479400000, 'corporateActions': [], 'preMarketTime': 1743508443, 'regularMarketTime': 1743451201, 'exchange': 'NMS', 'messageBoardId': 'finmb_24937', 'exchangeTimezoneName': 'America/New_York', 'exchangeTimezoneShortName': 'EDT', 'gmtOffSetMilliseconds': -14400000, 'market': 'us_market', 'esgPopulated': False, 'regularMarketChangePercent': 1.9412626, 'regularMarketPrice': 222.13, 'shortName': 'Apple Inc.', 'longName': 'Apple Inc.', 'priceHint': 2, 'preMarketChange': -2.1900024, 'preMarketChangePercent': -0.98591024, 'preMarketPrice': 219.94, 'regularMarketChange': 4.230011, 'regularMarketDayHigh': 224.97, 're

## Create the stock historical price tables

##### Upload the stock historical files from "sample_stock_hist_files" folder to your Unity catalog Volume. 
In real case these files can be sourced in various ways.

##### Then  run the below code to :
- Merge all the files in one spark data frame
- Then upload the themerged file in spark delta table which will be used in UC function (Agent Tool)

In [0]:
from pyspark.sql.functions import input_file_name, col, regexp_extract

# Function to rename columns to remove spaces and special characters
def rename_columns(df):
    return df.select([col(c).alias(c.replace(" ", "_").replace("Adj Close", "Adj_Close")) for c in df.columns])

# Load CSV files and add a column with the file name, then rename columns
tsla_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/TSLA_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 4))
    
)

aapl_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/AAPL_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 4))
)

googl_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/GOOGL_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 4))
)

jnj_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/JNJ_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 3))
)

jpm_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/JPM_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 3))
)

amzn_df = rename_columns(
    spark.read.option("header", "true")
    .csv("/Volumes/sarbanimaiti_catalog/agent_demo/stocks_historical_price/stock_hist_price/AMZN_prices.csv")
    .withColumn("stock_name", regexp_extract(input_file_name(), r'([^/]+)(?=\.)', 1).substr(0, 4))
)


# Union all dataframes
merged_df = tsla_df.union(aapl_df).union(googl_df).union(jnj_df).union(jpm_df).union(amzn_df)

# Write to a Delta table
merged_df.write.format("delta").mode("overwrite").saveAsTable("sarbanimaiti_catalog.agent_demo.merged_stock_prices")

In [0]:
%sql
-- verify the table 
SELECT DISTINCT stock_name FROM sarbanimaiti_catalog.agent_demo.merged_stock_prices

stock_name
JNJ
AAPL
JPM
AMZN
GOOGL
TSLA


## Create Agent Tool : Extract stock closing price from history table

In [0]:
%sql
--Drop the existing function to avoid conflicts
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.agent_demo.get_historical_closing_price;

In [0]:
%sql

-- This function takes stock ticker and duration in terms of year as input, refer the corresponding stock historical price table and fetch the historical closing price & volume of the stock for that duration. 
-- Duration in year  eg : '3'
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.get_hist_closing_price (
  stock_ticker STRING COMMENT 'Stock ticker will be passed in the query',
  yr_duration  BIGINT COMMENT 'Duration for which stock volume and closing price will be extracted'
  
)
returns table(Date DATE , Close FLOAT, Volume INT)
return
(
WITH FilteredStocks AS (
  SELECT
    Date,
    Close,
    Volume,
    stock_name
  FROM
    sarbanimaiti_catalog.agent_demo.merged_stock_prices
  WHERE
    stock_name = stock_ticker
    AND datediff(CURRENT_DATE(), Date) <= 800 * yr_duration
)
SELECT
  Date,
  Close,
  Volume
FROM
  FilteredStocks
ORDER BY
  Date ASC
)


#### Test the tool 

In [0]:

%sql
SELECT * FROM sarbanimaiti_catalog.agent_demo.get_hist_closing_price('JPM', 1)

Date,Close,Volume
2023-01-09,137.37,8482300
2023-01-10,138.6,9048700
2023-01-11,139.63,8703700
2023-01-12,139.49,11001700
2023-01-13,143.01,20200000
2023-01-17,140.8,14292900
2023-01-18,136.57,13392600
2023-01-19,134.75,13303400
2023-01-20,135.08,13092500
2023-01-23,137.27,10487100


## Create table to load customer investment preference data

Stesp:



In [0]:
%pip install Faker

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for Faker from https://files.pythonhosted.org/packages/64/82/f7d0c0a4ab512fd1572a315eec903d50a578c75d5aa894cf3f5cc04025e5/Faker-30.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--
   ━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.8 MB 4.8 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/1.8 MB 6.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/1.8 MB 7.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 1.0/1.8 MB 7.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 1.4/1.8 MB 8.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.8/1.8 MB 9.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.librar

In [0]:
import pandas as pd
import random
import json
from faker import Faker

# Initialize Faker for generating synthetic data
fake = Faker()

# Define ranges and options for generating data
income_ranges = ["low", "medium", "high"]
risk_tolerances = ["low", "medium", "high"]
investment_horizons = ["short-term", "medium-term", "long-term"]
investment_goals = ["retirement", "wealth accumulation", "education", "savings"]
industries = ["tech", "healthcare", "energy", "finance", "consumer goods"]
stock_types = ["growth", "dividend", "blue-chip", "value", "penny"]
trading_frequencies = ["daily", "weekly", "monthly"]
holding_periods = ["days", "weeks", "months", "years"]
special_requirements_options = ["ESG-focused investments", "no tobacco stocks", "", "", ""]

# Generate synthetic data
data = []
num_records = 100  # Number of records to generate

for _ in range(num_records):
    customer_id = fake.random_int(min=1000, max=9999)
    age = fake.random_int(min=18, max=75)
    gender = fake.random_element(elements=("Male", "Female"))
    income_range = fake.random_element(elements=income_ranges)
    risk_tolerance = fake.random_element(elements=risk_tolerances)
    investment_horizon = fake.random_element(elements=investment_horizons)
    investment_goal = fake.random_element(elements=investment_goals)
    preferred_industries = random.sample(industries, k=random.randint(1, 3))
    preferred_stock_type = fake.random_element(elements=stock_types)
    average_investment_amount = round(random.uniform(1000, 100000), 2)
    trading_frequency = fake.random_element(elements=trading_frequencies)
    holding_period = fake.random_element(elements=holding_periods)
    historical_performance = json.dumps({
        "avg_return": round(random.uniform(-10, 20), 2),
        "volatility": round(random.uniform(0.5, 3.0), 2)
    })
    special_requirements = fake.random_element(elements=special_requirements_options)
    
    # Add record to data list
    data.append([
        customer_id, age, gender, income_range, risk_tolerance, investment_horizon, 
        investment_goal, preferred_industries, preferred_stock_type, 
        average_investment_amount, trading_frequency, holding_period, 
        historical_performance, special_requirements
    ])

# Create DataFrame
columns = [
    "customer_id", "age", "gender", "income_range", "risk_tolerance", "investment_horizon",
    "investment_goals", "preferred_industries", "preferred_stock_types", 
    "average_investment_amount", "trading_frequency", "holding_period", 
    "historical_performance", "special_requirements"
]
df_synthetic = pd.DataFrame(data, columns=columns)

# Save to CSV if needed
#df_synthetic.to_csv("customer_stock_preferences_synthetic.csv", index=False)

print("Synthetic data generated and saved to 'customer_stock_preferences_synthetic.csv'.")


Synthetic data generated and saved to 'customer_stock_preferences_synthetic.csv'.


#### Upload Customer Stock Preferences data to spark delta table 

In [0]:
df_spark = spark.createDataFrame(df_synthetic)
df_spark.write.format("delta").mode("overwrite").saveAsTable("sarbanimaiti_catalog.agent_demo.customer_investment_preferences")

## Create Agent Tool : Extract Customer investment preference

In [0]:
%sql
-- Again check that it exists
DROP FUNCTION IF EXISTS sarbanimaiti_catalog.agent_demo.customer_investment_preferences;


In [0]:
%sql

-- This function takes stock ticker and duration in terms of year as input, refer the corresponding stock historical price table and fetch the historical closing price & volume of the stock for that duration. 
-- Duration in year  eg : '3'
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.cust_investment_pref (
  customerID BIGINT COMMENT 'Customer ID'
  
)
returns table(
  customer_id BIGINT, 
  age BIGINT,
  gender STRING, 
  income_range STRING, 
  risk_tolerance STRING, 
  investment_horizon STRING, 
  investment_goals STRING, 
  preferred_industries STRING, 
  preferred_stock_types STRING, 
  average_investment_amount DOUBLE, 
  trading_frequency BIGINT, 
  holding_period BIGINT, 
  historical_performance DOUBLE, 
  special_requirements STRING)
return
(
WITH CustomerPreference  AS (
  SELECT
    customer_id, 
    age,
    gender, 
    income_range, 
    risk_tolerance, 
    investment_horizon, 
    investment_goals, 
    preferred_industries, 
    preferred_stock_types, 
    average_investment_amount, 
    trading_frequency, 
    holding_period, 
    historical_performance, 
    special_requirements
  FROM
    sarbanimaiti_catalog.agent_demo.customer_investment_preferences
  WHERE
    customer_id = customerID )
  select * from CustomerPreference 
)


In [0]:
%sql
--test function
select * from sarbanimaiti_catalog.agent_demo.cust_investment_pref(3584)


customer_id,age,gender,income_range,risk_tolerance,investment_horizon,investment_goals,preferred_industries,preferred_stock_types,average_investment_amount,trading_frequency,holding_period,historical_performance,special_requirements
3584,30,Male,low,medium,long-term,wealth accumulation,"['energy', 'consumer goods']",blue-chip,83886.24,null,null,null,null
